## 问题描述

你需要使用 PINN 模型求解给定几何结构和流动参数的流体流动问题。

一个二维芯片放置在一个二维通道内。流体以 $u=1.5\text{ m/s}$ 的速度从入口进入，并从出口流出，出口压力为 $0 Pa$。所有其他壁面均视为无滑移。流动的运动粘度 $(\nu)$ 为 $0.02 \text{ }m^2/s$，密度 $(\rho)$ 为 $1 \text{ }kg/m^3$。问题如下图所示。

<img src="chip_2d_geom.png" alt="Drawing" style="width: 800px;"/>

## 练习

本题的主要目标是使用 PINN 模型正确地描述问题。为了实现这一目标，您必须成功完成以下部分：
1. 为问题定义正确的几何结构
2. 设置正确的边界条件和方程
3. 创建神经网络并求解问题

成功完成问题后，流动变量的分布应如下所示。此外，您还应力争使所有变量相对于给定 OpenFOAM 解的相对 $L_2$ 误差小于 0.2。

<img src="challenge_results.png" alt="Drawing" style="width: 650px;"/>

在此模板中，我们将为您提供一个框架代码，您需要填写该代码来定义和解决问题。

**注意：您需要编辑位于 `chip_2d/` 目录中的 [`chip_2d_template.py`](../../source_code/chip_2d/chip_2d_template.py) 脚本。配置文件已提供给您。**

首先，导入所需的软件包。请注意导入的各种模块和软件包，尤其是方程式、几何结构和架构。

```python
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import to_absolute_path, ModulusConfig, instantiate_arch
from modulus.utils.io import csv_to_dict
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.utils.sympy.functions import parabola
from modulus.eq.pdes.navier_stokes import NavierStokes
from modulus.eq.pdes.basic import NormalDotVec
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)

from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
```

现在，定义计算图的节点、适当的偏微分方程 (PDE)、模拟参数并生成几何结构。您将使用 PDEs 模块中的 `NavierStokes` 类来施加此问题的偏微分方程约束。

```python
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:
    
    #TODO: Replace all the placeholders with appropriate values
    # make list of nodes to unroll graph on
    ns = NavierStokes(nu=0.02, rho=1.0, dim=2, time=False)
    normal_dot_vel = NormalDotVec(["u", "v"])
    flow_net = instantiate_arch(
        input_keys=[placeholder],
        output_keys=[placeholder],
        cfg=cfg.arch.fourier_net,
    )
    nodes = (
        ns.make_nodes()
        + normal_dot_vel.make_nodes()
        + [flow_net.make_node(name="flow_network", jit=cfg.jit)]
    )
```

接下来，您将在本示例中使用二维几何模块。请为每个几何体填写相应的值。请记住，`Channel2D` 和 `Rectangle` 由其两个端点定义。Modulus 中通道和矩形的区别在于，通道几何体在 x 方向上没有边界曲线。这有助于获得 x 方向上无限的有向距离场。在某些湍流模型中，当有向距离场用作壁面距离时，这一点非常重要（更多详情请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/foundational/zero_eq_turbulence.html)）。因此，我们将使用 `Line` 几何体创建入口和出口（*请注意，这是一条二维线，而不是扩散教程中使用的 `Line1D` *）。

另请注意，`geo_lr` 和 `geo_hr` 是两个独立的几何体。它们的主要构造目的是改变几何体不同区域之间的采样。对不同区域的点采用不同的采样密度，使我们能够在靠近散热器（我们预计流场变化较大的地方）的位置采样更多点。实现这种可变采样的方法有很多，这里我们采用最简单的方法之一，即为高分辨率和低分辨率创建两个不同的几何体元素。

```python
    # add constraints to solver
    # simulation params
    channel_length = (-2.5, 2.5)
    channel_width = (-0.5, 0.5)
    chip_pos = -1.0
    chip_height = 0.6
    chip_width = 1.0
    inlet_vel = 1.5

    # define sympy variables to parametrize domain curves
    x, y = Symbol("x"), Symbol("y")
    
    #TODO: Replace x1, y1, x2, y2, and X's with appropriate values
    
    # define geometry
    channel = Channel2D(
        (channel_length[0], channel_width[0]), (channel_length[1], channel_width[1])
    )
    inlet = Line(
        (x1, y1),
        (x1, y2),
        normal=1,
    )
    outlet = Line(
        (x2, y1),
        (x2, y2),
        normal=1,
    )
    rec = Rectangle(
        (x1, y1),
        (x2, y2),
    )
    flow_rec = Rectangle(
        (chip_pos - 0.25, channel_width[0]),
        (chip_pos + chip_width + 0.25, channel_width[1]),
    )
    geo = channel - rec
    geo_hr = geo & flow_rec
    geo_lr = geo - flow_rec
```

当前问题是不可压缩流体的通道流。在这种情况下，通过通道每个横截面的质量流量以及体积流量都是恒定的。这可以作为问题的附加约束，有助于我们提高收敛速度。

尽可能地利用这些关于问题的额外知识，有助于更好、更快地解决问题。更多示例请参阅 [Modulus 用户文档](https://docs.nvidia.com/deeplearning/modulus/modulus-v2209/user_guide/foundational/scalar_transport.html)。

```python
    # Optional integral continuity planes to speed up convergence 
    x_pos = Symbol("x_pos")
    integral_line = Line((x_pos, channel_width[0]), (x_pos, channel_width[1]), 1)
    x_pos_range = {
        x_pos: lambda batch_size: np.full(
            (batch_size, 1), np.random.uniform(channel_length[0], channel_length[1])
        )
    }
```

现在，您将使用创建的几何体来定义问题的训练约束。实现以下所需的边界条件和方程。请记住，您必须为边界条件和减少方程残差创建约束。您可以参考 PDEs 模块中的 `NavierStokes` 类，了解方程的定义方式以及每个方程所需的键。

现在，基于此理解来定义问题。图中显示了入口边界条件的示例。此外，积分连续性约束已为您编写完成。

```python
    #TODO: Replace all the placeholders with appropriate values

    # make domain
    domain = Domain()

    # inlet
    inlet_parabola = parabola(y, channel_width[0], channel_width[1], inlet_vel)
    inlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=inlet,
        outvar={"u": inlet_parabola, "v": 0},
        batch_size=cfg.batch_size.inlet,
    )
    domain.add_constraint(inlet, "inlet")

    # outlet
    outlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=outlet,
        outvar={placeholder},
        batch_size=cfg.batch_size.outlet,
        criteria=Eq(x, channel_length[1]),
    )
    domain.add_constraint(outlet, "outlet")

    # no slip
    no_slip = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={placeholder},
        batch_size=cfg.batch_size.no_slip,
    )
    domain.add_constraint(no_slip, "no_slip")

    # interior lr
    interior_lr = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo_lr,
        outvar={placeholder},
        batch_size=cfg.batch_size.interior_lr,
        bounds={x: channel_length, y: channel_width},
        lambda_weighting={placeholder},
    )
    domain.add_constraint(interior_lr, "interior_lr")

    # interior hr
    interior_hr = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo_hr,
        outvar={placeholder},
        batch_size=cfg.batch_size.interior_hr,
        bounds={x: channel_length, y: channel_width},
        lambda_weighting={placeholder},
    )
    domain.add_constraint(interior_hr, "interior_hr")

    # integral continuity
    def integral_criteria(invar, params):
        sdf = geo.sdf(invar, params)
        return np.greater(sdf["sdf"], 0)

    integral_continuity = IntegralBoundaryConstraint(
        nodes=nodes,
        geometry=integral_line,
        outvar={"normal_dot_vel": 1},
        batch_size=cfg.batch_size.num_integral_continuity,
        integral_batch_size=cfg.batch_size.integral_continuity,
        lambda_weighting={"normal_dot_vel": 1},
        criteria=integral_criteria,
        parameterization=x_pos_range,
    )
    domain.add_constraint(integral_continuity, "integral_continuity")
```

现在，向问题添加验证数据。`openfoam` 目录包含使用 OpenFOAM 求解器求解同一问题的解决方案。CSV 文件会被读取并转换为字典。现在，您需要完成验证器的定义。

```python
    #TODO: Set the appropriate normalization for the validation data
    # The validation data has domain extents of (0,0) to (5,1). Normalize this based on your definition of the domain

    # add validation data
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path("openfoam/2D_chip_fluid0.csv"), mapping)
    openfoam_var["x"] -= placeholder  # TODO normalize pos
    openfoam_var["y"] -= placeholder  # TODO normalize pos
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]
    }
    openfoam_validator = PointwiseValidator(
        invar=placeholder, true_outvar=placeholder, nodes=nodes
    )
    domain.add_validator(openfoam_validator)
```

现在，通过创建求解器来解决问题，完成代码的最后一部分。配置文件中已为您定义好该问题的重要超参数。您可以随意调整这些超参数，并观察其对结果和收敛速度的影响。

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

完成模板脚本 ([`chip_2d_template.py`](../../source_code/chip_2d/chip_2d_template.py)) 中的所有代码部分后，您可以像我们在之前的教程中看到的那样执行它。记录相对于 OpenFOAM 解的相对 L2 误差，并尝试使所有变量的误差都低于 0.2。还可以尝试通过读取在网络检查点中创建的 `.npz` 文件，使用等值线图来可视化您的结果。